Решение задачи кластеризации с использованием трех методов:
  1. KMeans
  2. DBSCAN
  3. Spectral Clustering

Импорт библиотек и загрузка исходных данных

In [2]:
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


Подготовка данных

In [3]:
processed_data = data.copy()

categorical_encoders = {}
for feature in processed_data.select_dtypes(include='object').columns:
    if feature != 'income':
        encoder = LabelEncoder()
        processed_data[feature] = encoder.fit_transform(processed_data[feature])
        categorical_encoders[feature] = encoder

scaler = StandardScaler()
numerical_features = processed_data.select_dtypes(include=['int64', 'float64']).columns
processed_data[numerical_features] = scaler.fit_transform(processed_data[numerical_features])

Реализация методов кластеризации

In [4]:
kmeans = KMeans(n_clusters=5, random_state=45)
kmeans_labels = kmeans.fit_predict(processed_data.drop(columns=['income']))
kmeans_num_clusters = len(set(kmeans_labels))
print(f"KMeans: Число кластеров = {kmeans_num_clusters}")
ari_kmeans = adjusted_rand_score(LabelEncoder().fit_transform(data['income']), kmeans_labels)
print(f'Adjusted Rand Index для KMeans: {ari_kmeans}')

dbscan = DBSCAN(eps=1.5, min_samples=55)
dbscan_labels = dbscan.fit_predict(processed_data.drop(columns=['income']))
dbscan_num_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
print(f"DBSCAN: Число кластеров = {dbscan_num_clusters}")
ari_dbscan = adjusted_rand_score(LabelEncoder().fit_transform(data['income']), dbscan_labels)
print(f'Adjusted Rand Index для DBSCAN: {ari_dbscan}')

gmm = GaussianMixture(n_components=5, random_state=45)
gmm_labels = gmm.fit_predict(processed_data.drop(columns=['income']))
gmm_num_clusters = len(set(gmm_labels))
print(f"Gaussian Mixture: Число кластеров = {gmm_num_clusters}")
ari_gmm = adjusted_rand_score(LabelEncoder().fit_transform(data['income']), gmm_labels)
print(f'Adjusted Rand Index для Gaussian Mixture: {ari_gmm}')

KMeans: Число кластеров = 5
Adjusted Rand Index для KMeans: 0.021722873034472883
DBSCAN: Число кластеров = 7
Adjusted Rand Index для DBSCAN: -0.01741673808244024
Gaussian Mixture: Число кластеров = 5
Adjusted Rand Index для Gaussian Mixture: 0.1507825200893089


Выводы

* KMeans: Метод кластеризации с 5 кластерами показал значение Adjusted Rand Index (0.0217), что указывает на слабую связь между результатами кластеризации и истинными метками. Это свидетельствует о том, что KMeans не смог точно разделить данные на смысловые группы. Возможно, результат можно улучшить, если пересмотреть выбор признаков или применить другие методы предобработки данных, так как текущий результат не дает четкой структуры.

* DBSCAN:  В случае с DBSCAN, который обнаружил 7 кластеров, значение Adjusted Rand Index составило (-0.0174), что также указывает на слабую согласованность кластеров с истинными метками. Этот результат говорит о том, что DBSCAN не смог достоверно выделить смысловые группы в данных.

* Gaussian Mixture: С 5 кластерами метод Gaussian Mixture показал Adjusted Rand Index (0.1508), что является лучшим результатом среди всех трех методов, но всё еще довольно низким. Этот показатель говорит о том, что хотя метод смог выделить некоторые структуры в данных, результаты всё еще далеки от идеала.

Все три метода кластеризации (KMeans, DBSCAN и Gaussian Mixture) продемонстрировали относительно слабую согласованность с истинными метками. Это может свидетельствовать о том, что данные не обладают четкой внутренней структурой, либо выбранные методы и параметры не оптимальны для данной задачи. Стоит продолжить эксперименты с настройками алгоритмов, улучшить представление данных и, возможно, попробовать другие методы кластеризации, чтобы добиться более точных результатов.